In [1]:
if True:
    from unsloth import FastVisionModel
    model, tokenizer = FastVisionModel.from_pretrained(
        model_name = "VAT_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = True, # Set to False for 16bit LoRA
    )
    FastVisionModel.for_inference(model) # Enable for inference!

import re, json, ast
from typing import Tuple, Any, List, Optional

def repair_json(s: str, schema: Optional[dict] = None) -> Tuple[str, Any, List[str]]:
    """
    將「幾乎 JSON」的字串修復成合法 JSON。
    回傳: (fixed_text, obj, logs)
      fixed_text: 修復後的 JSON 字串
      obj:        對應的 Python 物件 (dict/list)
      logs:       修復步驟紀錄
    """
    logs: List[str] = []
    text = s.strip()

    # 0) 去掉 ```json ... ``` 或一般 ``` 區塊外殼
    if "```" in text:
        text = re.sub(r"```(?:json|JSON)?", "", text)
        text = text.replace("```", "")
        text = text.strip()
        logs.append("removed code fences")

    # 1) 只擷取最外層 {...} 或 [...]
    def _extract_json_region(t: str) -> str:
        lb, rb = t.find("{"), t.rfind("}")
        if lb != -1 and rb != -1 and rb > lb:
            return t[lb:rb+1]
        lb, rb = t.find("["), t.rfind("]")
        if lb != -1 and rb != -1 and rb > lb:
            return t[lb:rb+1]
        return t

    text2 = _extract_json_region(text)
    if text2 != text:
        logs.append("extracted outer JSON-like region")
        text = text2

    # 2) 嘗試標準 JSON
    try:
        obj = json.loads(text)
        logs.append("parsed by json")
        if schema:
            from jsonschema import validate
            validate(obj, schema); logs.append("validated by jsonschema")
        return json.dumps(obj, ensure_ascii=False, indent=2), obj, logs
    except Exception as e:
        logs.append(f"json.loads failed: {e}")

    # 3) 用 ast.literal_eval 吃單引號/尾逗號 等 Python 字面量
    try:
        obj = ast.literal_eval(text)
        logs.append("parsed by ast.literal_eval")
        if schema:
            from jsonschema import validate
            validate(obj, schema); logs.append("validated by jsonschema")
        return json.dumps(obj, ensure_ascii=False, indent=2), obj, logs
    except Exception as e:
        logs.append(f"ast.literal_eval failed: {e}")

    # 4) 常見修補：彎引號→直引號、True/False/None→JSON、刪尾逗號、單引號→雙引號
    fixed = text.translate(str.maketrans({
        "\u201c": '"', "\u201d": '"', "\u2018": "'", "\u2019": "'",
    }))
    if fixed != text:
        logs.append("normalized curly quotes")

    # Python 常量 → JSON 常量
    fixed2 = re.sub(r'(?<!")\bTrue\b(?!")', "true", fixed)
    fixed2 = re.sub(r'(?<!")\bFalse\b(?!")', "false", fixed2)
    fixed2 = re.sub(r'(?<!")\bNone\b(?!")', "null", fixed2)
    if fixed2 != fixed:
        logs.append("converted Python literals to JSON")
    fixed = fixed2

    # 移除尾逗號
    no_trailing_commas = re.sub(r",\s*([}\]])", r"\1", fixed)
    if no_trailing_commas != fixed:
        logs.append("removed trailing commas")
    fixed = no_trailing_commas

    # 粗略：單引號 → 雙引號（在中文內容通常安全）
    dq = re.sub(r"(?<!\\)'", '"', fixed)
    if dq != fixed:
        logs.append("replaced single quotes with double quotes")
    fixed = dq

    # 5) 最終嘗試標準 JSON
    try:
        obj = json.loads(fixed)
        logs.append("fixed manually then parsed by json")
        if schema:
            from jsonschema import validate
            validate(obj, schema); logs.append("validated by jsonschema")
        return json.dumps(obj, ensure_ascii=False, indent=2), obj, logs
    except Exception as e:
        logs.append(f"final json.loads failed: {e}")
        # 兜底：包 raw
        fallback = {"raw": s}
        if schema:
            logs.append("returned raw because schema validation/parse failed")
        return json.dumps(fallback, ensure_ascii=False, indent=2), fallback, logs

c:\Users\user\anaconda3\envs\VAT\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Exception in thread Thread-4 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\user\anaconda3\envs\VAT\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "c:\Users\user\anaconda3\envs\VAT\Lib\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\user\anaconda3\envs\VAT\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\user\anaconda3\envs\VAT\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "<frozen codecs>", line 322, in decode
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa4 in position 7: invalid start byte


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


W0916 12:07:13.500000 39828 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


🦥 Unsloth Zoo will now patch everything to make training faster!


c:\Users\user\anaconda3\envs\VAT\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:339: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:35.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"{DEVICE_TYPE}:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.9.4: Fast Qwen2_5_Vl patching. Transformers: 4.56.1.
   \\   /|    NVIDIA GeForce RTX 4070. Num GPUs = 1. Max memory: 11.994 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [4]:
from pathlib import Path
import json
from PIL import Image

FastVisionModel.for_inference(model)  # inference 模式

image = Image.open("./invoice2.jpg").convert("RGB")

instruction = "你是發票/單據分類器與結構化抽取器，請辨識這張文件"

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]

# 準備輸入
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")

# 產生（不使用 streamer，改成一次取回）
gen_ids = model.generate(
    **inputs,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.1,
    min_p=0.1,
    do_sample=True,              # 若你想要可重現，可改成 False
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)

# 只取「模型新產生」的 token，排除提示部分
prompt_len = inputs["input_ids"].shape[1]
new_token_ids = gen_ids[0, prompt_len:]

output_text = tokenizer.decode(new_token_ids, skip_special_tokens=True).strip()

try:
    result = json.loads(output_text)
except Exception:
    result, obj, logs = repair_json(output_text)

print(result)


{
  "gt_parse": {
    "Other": [
      "柏格文具禮品股份有限公司",
      "統編",
      "710台南市永康區崑山街183巷23號"
    ],
    "Tail": {
      "SalesTax": "475",
      "SalesTotalAmount": "9498",
      "TotalAmount": "9973"
    },
    "body": {
      "BuyerTaxIDNumber": "53812386",
      "CompanyTaxIDNumber": "28652798",
      "InvoiceDay": "15",
      "InvoiceMonth": "12",
      "InvoiceYear": "110",
      "PhoneNumber": "06-2059968"
    },
    "header": {
      "InvoiceNumber": "23747161",
      "PrefixTwoLetters": "TC"
    }
  }
}


(old inference)

In [ ]:
from PIL import Image

image = Image.open("./invoice.jpg").convert("RGB")
instruction = "你是發票/單據分類器與結構化抽取器，請辨識這張文件"

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 256,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

{'gt_parse': {'Other': ['柏格文具禮品股份有限公司', '台南市永康區崙山街183巷23號', '統編', '710'], 'Tail': {'SalesTax': '475', 'SalesTotalAmount': '9498', 'TotalAmount': '9973'}, 'body': {'BuyerTaxIDNumber': '53812386', 'CompanyTaxIDNumber': '28652798'}, 'header': {'InvoiceDay': '15', 'InvoiceMonth': '12', 'InvoiceYear': '110', 'PrefixTwoLetters': 'TC'}}}<|im_end|>
